In [ ]:
import os
import re
import pandas as pd

# Caminho dos arquivos
base_path = {'s5': '../data_raw/s5Bruto','s6': '../data_raw/s6Bruto'}
saida_excel = "../data_raw/saida_bancos.xlsx"
log_path = "../data_raw/log_bancos.txt"

# Regex para capturar linhas válidas da tabela
line_regex = re.compile(
    r'^\s*([\w_]+)\s+\|\s+\w+\s+\|\s+\w+\s+\|\s+\S+\s+\|\s+\S+\s+\|\s+\S*\s+\|\s+([\d\.]+)\s*(MB|GB|kB)\s+\|\s+\w+\s+\|',
    re.IGNORECASE
)

dados = []
arquivos_processados = []

def size_to_mb(size, unit):
    size = float(size.replace(',', ''))
    if unit.upper() == 'GB':
        return size * 1024
    elif unit.upper() == 'MB':
        return size
    elif unit.upper() == 'KB':
        return size / 1024
    return size

for filename in os.listdir(base_path):
    if filename.startswith("bancos_") and filename.endswith(".txt"):
        # Extrai data e servidor do nome do arquivo
        match = re.match(r'bancos_(\d{8})_s(\d)\.txt', filename)
        if not match:
            arquivos_processados.append(f'❌ {filename} - Nome fora do padrão')
            continue
        data_raw, servidor = match.groups()
        data_formatada = f"{data_raw[6:8]}/{data_raw[4:6]}/{data_raw[:4]}"
        try:
            with open(os.path.join(base_path, filename), 'r') as file:
                for line in file:
                    m = line_regex.match(line)
                    if m:
                        base, size, unit = m.groups()
                        tamanho_mb = size_to_mb(size, unit)
                        tipo = "arq" if base.startswith("arq_") else "normal"
                        dados.append({
                            "Data": data_formatada,
                            "Servidor": f"s{servidor}",
                            "Base": base,
                            "Tamanho (MB)": round(tamanho_mb, 2),
                            "Tipo": tipo
                        })
            arquivos_processados.append(f'✅ {filename}')
        except Exception as e:
            arquivos_processados.append(f'❌ {filename} - Erro: {e}')

# Salvar log de arquivos
with open(log_path, 'w') as log_file:
    log_file.write('\n'.join(arquivos_processados))

# Criar DataFrame e processar os dados
df = pd.DataFrame(dados)

# Agrupamento geral
df_agrupado = df.groupby(['Data', 'Servidor', 'Base', 'Tipo'], as_index=False)['Tamanho (MB)'].sum()
df_ordenado = df_agrupado.sort_values(by=['Base', 'Data'])

# Totais por Base e Servidor
df_totais = df_ordenado.groupby(['Servidor', 'Base', 'Tipo'], as_index=False)['Tamanho (MB)'].sum()

# Tamanhos separados por tipo
df_arq = df[df['Tipo'] == 'arq'].groupby(['Servidor', 'Base', 'Data'], as_index=False)['Tamanho (MB)'].sum()
df_normal = df[df['Tipo'] == 'normal'].groupby(['Servidor', 'Base', 'Data'], as_index=False)['Tamanho (MB)'].sum()

# Crescimento por Base e Data (apenas arquivos normais)
df_crescimento = df_normal.copy()
df_crescimento = df_crescimento.sort_values(['Servidor', 'Base', 'Data'])
df_crescimento['Crescimento (%)'] = df_crescimento.groupby(['Servidor', 'Base'])['Tamanho (MB)'].pct_change() * 100
df_crescimento['Diferença (MB)'] = df_crescimento.groupby(['Servidor', 'Base'])['Tamanho (MB)'].diff()

# Exportar tudo em um único Excel com múltiplas abas
with pd.ExcelWriter(saida_excel, engine='openpyxl') as writer:
    df_ordenado.to_excel(writer, sheet_name='Resumo por Base', index=False)
    df_totais.to_excel(writer, sheet_name='Totais por Base', index=False)
    df_crescimento.to_excel(writer, sheet_name='Crescimento (%)', index=False)
    df_arq.to_excel(writer, sheet_name='Apenas arq', index=False)
    df_normal.to_excel(writer, sheet_name='Apenas normal', index=False)

print(f'✅ Relatório completo exportado para: {saida_excel}')

TypeError: listdir: path should be string, bytes, os.PathLike, integer or None, not dict